## Visualizing Spatial Data with Pandas and Bokeh

[bokeh](http://bokeh.pydata.org/en/latest/) is a relatively new JavaScript visualization language for Python that is modeled after D3 but is intended to be able to handle millions of data points.

>Bokeh is a Python interactive visualization library that targets modern web browsers for presentation. Its goal is to provide elegant, concise construction of novel graphics in the style of D3.js, and to extend this capability with high-performance interactivity over very large or streaming datasets. Bokeh can help anyone who would like to quickly and easily create interactive plots, dashboards, and data applications. ([Bokeh Website](http://bokeh.pydata.org/en/latest/))

The advantage of Bokeh over matplotlib is that the visualizations can be interactive (via the JavaScript).

From the U.K. accident data, we can plot the location of accidents for which latitude and longitude values are provided.

In [1]:
import os
import sqlite3 as sqlite
DATADIR = os.path.join(os.path.expanduser("~"),"DATA",
                       "Misc")
print(os.path.exists(DATADIR))
import pandas as pd
import numpy as np

True


In [2]:
from bokeh.io import output_notebook

### This enables drawing directly in the notebook

In [3]:
output_notebook()

Loading BokehJS ...

### Read in the data

In [4]:
data = pd.read_csv(os.path.join(DATADIR,
                         "Accidents7904.csv"),
                   usecols = ['Longitude',"Latitude","Date"]).dropna()
data

Longitude   Latitude        Date
4883216  -0.271752  51.715661  25/12/1999
4883217  -0.239977  51.695136  17/12/1999
4883218  -0.270037  51.715096  15/12/1999
4883219  -0.263233  51.711309  02/12/1999
4883220  -0.227225  51.688200  04/12/1999
4883221  -0.375451  51.690074  29/12/1999
4883222  -0.279194  51.717928  17/12/1999
4883223  -0.384303  51.664928  17/12/1999
4883224  -0.372406  51.674116  17/11/1999
4883225  -0.241739  51.698220  23/11/1999
4883226  -0.241470  51.697677  26/11/1999
4883227  -0.256434  51.707432  29/11/1999
4883228  -0.279706  51.715868  24/11/1999
4883229  -0.242572  51.699132  22/11/1999
4883230  -0.232021  51.687643  12/11/1999
4883231  -0.225064  51.691675  25/11/1999
4883232  -0.221562  51.703492  02/11/1999
4883233  -0.383738  51.664561  13/11/1999
4883234  -0.235373  51.690751  20/10/1999
4883235  -0.262675  51.710761  11/10/1999
4883236  -0.236494  51.691667  01/10/1999
4883237  -0.231337  51.686644  02/09/1999
4883238  -0.238311  51.693313  18/09/1999
4883239  -0.252890  51.705491  20/09/1999
4883240  -0.241343  51.697225  23/09/1999
4883241  -0.384323  51.664389  04/09/1999
4883242  -0.279194  51.717928  25/09/1999
4883243  -0.033236  51.727647  20/09/1999
4883244  -0.241739  51.698220  14/08/1999
4883245  -0.254027  51.706047  19/07/1999
...            ...        ...         ...
6224168  -3.271193  55.062765  09/10/2004
6224169  -3.391062  55.029011  15/10/2004
6224170  -3.470793  55.325816  19/10/2004
6224171  -3.232324  54.995313  21/10/2004
6224172  -3.167246  55.090497  21/10/2004
6224173  -3.263876  55.029320  22/10/2004
6224174  -3.250516  55.012293  28/10/2004
6224175  -3.459391  55.315798  28/10/2004
6224176  -3.113030  55.124256  06/11/2004
6224177  -3.196337  55.046718  08/11/2004
6224178  -2.967089  55.262815  11/11/2004
6224179  -3.446287  55.276142  12/11/2004
6224180  -3.444244  55.332604  12/11/2004
6224181  -3.067366  54.993727  25/11/2004
6224182  -2.999541  55.151301  27/11/2004
6224183  -3.192532  55.060865  29/11/2004
6224184  -3.284680  54.994951  29/11/2004
6224185  -3.255347  54.986720  02/12/2004
6224186  -3.176264  54.986611  03/12/2004
6224187  -3.498349  55.107545  09/12/2004
6224188  -3.279674  54.985028  09/12/2004
6224189  -3.333805  55.193845  10/12/2004
6224190  -3.405502  55.182165  12/12/2004
6224191  -3.444060  55.331887  17/12/2004
6224192  -3.261148  54.987289  18/12/2004
6224193  -3.215400  54.993955  20/12/2004
6224194  -3.285649  54.995929  22/12/2004
6224195  -3.511052  55.356510  27/12/2004
6224196  -3.476987  55.331314  27/12/2004
6224197  -3.250229  55.013104  28/12/2004

[1337067 rows x 3 columns]

####  We can use the [``sample``](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html) method to get a subset of DataFrame

In [5]:
subdata = data.sample(2000)
mean_long = np.mean(subdata['Longitude'])
mean_lat  = np.mean(subdata['Latitude'])


In [6]:
from bokeh.io import output_file, show
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, DataRange1d, 
    PanTool, WheelZoomTool, BoxSelectTool, HoverTool
)

hover = HoverTool()
map_options = GMapOptions(lat=mean_lat, 
                          lng=mean_long, 
                          map_type="roadmap", zoom=6)

plot = GMapPlot(
    x_range=DataRange1d(), 
    y_range=DataRange1d(), 
    map_options=map_options
)
plot.title.text = "U.K. Road Accidents"

source = ColumnDataSource(
    data=dict(
        lat=subdata['Latitude'],
        lon=subdata['Longitude'],
    )
)


hover.tooltips.append(('index','$index'))
circle = Circle(x="lon", y="lat", size=2, 
                fill_color="blue", fill_alpha=0.8, 
                line_color=None)
plot.add_glyph(source, circle)

plot.add_tools(PanTool(),WheelZoomTool(), BoxSelectTool(), hover)
show(plot)